# Imports

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# OpenAI API Key

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


# About...

In this project I use the code from day 1 for something I do at work. I'm a real estate appraiser and when I prepare a valuation for some real estate, I analyze the local market, and in particular the city where the property is located. I then gather economy-related information and create a report from it. I'm based in Poland, so the report is in Polish. Here, I want to ask the model to make such a report for me, using the official website of the city and its related Wikipedia article.

# The Code

In [3]:
openai = OpenAI()

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
system_prompt = """You are an analyst and real estate appraiser who checks out the official websites 
of cities as well as articles related to these cities on Wikipedia, searching the particular pages 
of the official website and the Wikipedia article for economic data, in particular the 
demographic structure of the city, its area, and how it's subdivided into built-up area, 
rural area, forests, and so on, provided this kind of information is available. 
The most important information you want to find is that related to the real estate market in the city, 
but also the general economy of the city, so what kind of factories or companies there are, commerce, 
business conditions, transportation, economic growth in recent years, and recent investments. 
wealth of the inhabitants, and so on, depending on what kind of information is available on the website. 
Combine the information found on the official website with the information found on Wikipedia, and in case
of discrepancies, the official website should take precedence. If any of the information is missing,
just omit it entirely and don't mention that it is missing, just don't write about it at all.
When you gather all the required information, create a comprehensive report presenting 
the data in a clear way, using markdown, in tabular form where it makes sense. 
The length of the report should be about 5000 characters. And one more thing, the report should be entirely 
in Polish. """

In [6]:
def user_prompt_for(city, city_website, wiki_website):
    user_prompt = f"You are looking at the official website of the city {city}, and its wiki article."
    user_prompt += f"\nThe contents of this website is as follows: \
please provide a comprehensive report of economy-related data for the city of {city}, available on the \
particular pages and subpages of its official website and Wikipedia in markdown. \
Add tables if it makes sense for the data. The length of the report should be about 5000 characters. \
The report should be in Polish.\n\n"
    user_prompt += city_website.text
    user_prompt += wiki_website.text
    return user_prompt

In [7]:
def messages_for(city, city_website, wiki_website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(city, city_website, wiki_website)}
    ]

In [8]:
def report(url_official, url_wiki, city):
    city_website = Website(url_official)
    wiki_website = Website(url_wiki)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(city, city_website, wiki_website)
    )
    return response.choices[0].message.content

# Raw Result

In [9]:
report("https://www.rudaslaska.pl/", "https://pl.wikipedia.org/wiki/Ruda_%C5%9Al%C4%85ska", "Ruda Śląska")

'# Raport Gospodarczy - Ruda Śląska\n\n## 1. Podstawowe informacje\n\n| Parametr                            | Wartość                      |\n|-------------------------------------|-------------------------------|\n| Powierzchnia                        | 77,64 km²                    |\n| Liczba ludności (2024)             | 129 531                      |\n| Gęstość zaludnienia                 | 1668,4 os./km²               |\n| Dzielnice                           | 11 (m.in. Ruda, Wirek, Halemba, Chebzie, Godula, Orzegów) |\n\n## 2. Gospodarka\n\nRuda Śląska, zlokalizowana w centrum Górnośląskiego Okręgu Przemysłowego, jest ważnym ośrodkiem przemysłowym z rozbudowanym sektorem przemysłowym, w tym:\n\n### 2.1. Główne zakłady przemysłowe\n\n| Zakład                         | Typ działalności              | Status              |\n|--------------------------------|--------------------------------|---------------------|\n| KWK Ruda                       | Kopalnia węgla kamiennego    | Czyn

# Polished Result

In [10]:
def display_report(url_official, url_wiki, city):
    rep = report(url_official, url_wiki, city)
    display(Markdown(rep))

In [11]:
display_report("https://www.rudaslaska.pl/", "https://pl.wikipedia.org/wiki/Ruda_%C5%9Al%C4%85ska", "Ruda Śląska")

# Raport dotyczący ekonomii Rudy Śląskiej

## Informacje ogólne

| Właściwość            | Wartość                        |
|----------------------|-------------------------------|
| Powierzchnia         | 77,64 km²                     |
| Liczba ludności      | 129 531 (stan na 30.06.2024) |
| Gęstość zaludnienia  | 1668,4 os./km²                |
| Władze               | Prezydent: Michał Pierończyk  |

Ruda Śląska to miasto w województwie śląskim, położone w centrum Górnośląskiego Okręgu Przemysłowego. Jest częścią konurbacji górnośląskiej i wyróżnia się silnym przemysłem oraz historycznym znaczeniem w zakresie wydobycia węgla i metali.

## Struktura demograficzna

- Największa populacja została odnotowana w 1991 r. (171 645 mieszkańców).
- Spadek liczby ludności trwa od lat 90.
  
## Gospodarka

Ruda Śląska ma silne fundamenty przemysłowe. W mieście działają zarówno zakłady górnicze, jak i firmy z sektora energetycznego oraz małych i średnich przedsiębiorstw. Wspierane są przez lokalne agencje rozwoju oraz Inkubatory Przedsiębiorczości.

| Zakład przemysłowy                   | Status             |
|--------------------------------------|--------------------|
| KWK Ruda                              | Czynna             |
| KWK Halemba                           | Zlikwidowana w 2016 |
| Huta Pokój                            | Czynna             |
| Elektrociepłownia Mikołaj            | Czynna             |
| ZEKON                                 | Czynna             |
| STAMA                                 | Czynna             |

**Inwestycje i wsparcie dla przemysłu:**

- **Rudzka Agencja Rozwoju „Inwestor”** – wspiera lokalnych przedsiębiorców.
- **Rudzki Inkubator Przedsiębiorczości** – oferuje szkolenia i doradztwo.
- **Śląski Park Przemysłowy** – współpraca z Sąsiednimi Gminami na rozwój terenów poprzemysłowych.

## Transport

Ruda Śląska jest dobrze skomunikowana z innymi miastami dzięki rozwiniętej infrastrukturze transportowej.

### Transport drogowy

- **Drogowa Trasa Średnicowa** – kluczowa arteria w północnej części miasta.
- **Autostrada A4** – zapewnia szybki dostęp do południowych regionów Polski oraz Niemiec.

### Transport publiczny

| Typ transportu       | Rodzaje           |
|----------------------|-------------------|
| Autobusy             | 27 linii          |
| Tramwaje             | 4 linie           |
| Połączenia kolejowe  | Ruda Chebzie, Ruda Śląska |

## Ogólny stan gospodarki

W ostatnich latach w mieście nie odnotowano znaczącej poprawy w liczbie mieszkańców, ale lokalne władze inwestują w rewitalizację przestrzeni miejskiej oraz poprawę warunków dla przedsiębiorców. Gospodarka miasta przechodzi transformację, starając się zróżnicować swoje sektory przemysłowe, by nie polegać wyłącznie na górnictwie.

### Wydatki i inwestycje

Wśród głównych inwestycji znajduje się rewitalizacja terenów po przemysłowych oraz projekty związane z zieloną energią w ramach Programu Czyste Powietrze.

## Wnioski
Ruda Śląska to miasto z bogatą historią przemysłową, które w obliczu współczesnych wyzwań stara się dostosować swoją gospodarkę do bieżących potrzeb. Podjęte inwestycje, zwłaszcza w innowacyjność i małe przedsiębiorstwa, mogą pomóc w dalszym rozwoju miasta i polepszeniu jakości życia mieszkańców. Miasto, mimo spadku populacji, wciąż stanowi ważny punkt na gospodarczej mapie Śląska.